# CX967


There are two slits CX967 and CX972. 




![](images/both.png)

![](images/ssem.png)



From the 2D spectrum and the preimage I think that **CX967** is the first section from **0 to 70**.

- - - 


## Information

# Spectrum extraction

First import the require packages

In [1]:
from astropy.io import fits
import os
from stsci.tools import capable
capable.OF_GRAPHICS = False
from pyraf import iraf
import numpy as np
from shutil import copyfile
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Span, Label, Arrow, NormalHead
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation
from bokeh.layouts import  column, row
from bokeh.palettes import viridis, BrBG8, Accent8, OrRd8, Pastel2_4, Category20b_8, Viridis8,Dark2_8,Paired8, Set1_4
import re
output_notebook()

Created directory /root/VIMOSReduced/cx0967/pyraf for cache


Loading BokehJS ...

## Defining the extraction parameters

First we define the parameters to extract the desire spectra:
    - Name of the extracted spectrum
    - Center, low and upper limits of the aperture
    - Background sample. Note that the background coordinates are relative to the aperture center and not image pixel coordinates so the endpoints need not be integer.
    - 2D spectra to use. Can use the SSEM or SEXM. The SSEM doesnt have the sky substracted, and the background substraction can be done with IRAF and apall. SEXM has the background substracted by the VIMOS pipeline so background can be set to none
    
To check what values can the website: https://vimos.manuelpm.me/

In [2]:
sourcename = 'cx967sky'
databasegeneral = 'database/apcx967sexm'
filename = 'database/ap'+sourcename
fitsfilename = 'mos_science_sky_extracted_Q3.fits'

In [3]:
ape = fits.open(fitsfilename)
##For srfm[0].header["CTYPE1"] = 'LINEAR'
xn = ape[0].header["NAXIS1"]
refx = ape[0].header["CRVAL1"]
step = ape[0].header['CD1_1']
cr = ape[0].header['CRPIX1']
ape = ape[0].data
#
xlist = [ refx + step*(i - cr) for i in np.arange(ape.shape[1] +1) ]
#

#Default is half
y = ape[:,int(ape.shape[1]/2)]
x = list(range(y.shape[0]))
c2 = figure(x_axis_label='index',title='Wavelength',toolbar_location="above",active_scroll='wheel_zoom')
source2 = ColumnDataSource(data=dict(x=x,y=y))
sepaape = 5
sourceall2 = ColumnDataSource(data= dict([ (str(int(xlist[i])), ape[:,i]  ) for i in list(range(0,ape.shape[1],sepaape)) ]))

wavelist =  [ int(xlist[i]) for i in list(range(0,ape.shape[1],sepaape)) ]
sepawave = wavelist[1]- wavelist[0]


c2.line('x','y', source=source2)

callback2 = CustomJS(args=dict(source2 = source2, sourceall2 = sourceall2 ), code="""

        var data2 = source2.data;
        var data22 = sourceall2.data;
        var f = cb_obj.value;
        y = data2['y'];
        y2 = data22[f.toString()];


        for (i = 0; i < y.length; i++) {
            y[i] = y2[i];
        }
        source2.trigger('change');
    """)



slider2 = Slider(title="Wavelength", value=wavelist[int(len(wavelist)/2)], 
                 start=wavelist[0], end=wavelist[-1], step=sepawave,callback=callback2)


hover2 = HoverTool(
        tooltips=[
#            ("index", "$index"),
            ("(x,y)", "($x{1}, $y)"),
        ]
    )
c2.add_tools(hover2)

layout = column(slider2,c2)
show(layout)

## Aperture selection

Select the center the lower and upper limit, as well as the region for the background

In [4]:
center = 49
low = -5 
high = 5
b_low = [36-center,40-center]
b_up = [70-center,61-center]

## Exposure time

First we multiply the 2D spectra times the exposure time

In [5]:
spectrawithsky = fits.open(fitsfilename)
exptime = spectrawithsky[0].header['EXPTIME']
iraf.stsdas()
iraf.images.imutil()
if os.path.exists(sourcename+'.fits'):
    os.remove(sourcename+'.fits')
    
iraf.images.imutil.imarith(fitsfilename,'*', exptime,sourcename)



      +------------------------------------------------------------+
      |       Space Telescope Science Data Analysis System         |    
      |                   STSDAS Version 3.17                      |
      |                                                            |
      |   Space Telescope Science Institute, Baltimore, Maryland   |
      |   Copyright (C) 2014 Association of Universities for       |
      |            Research in Astronomy, Inc.(AURA)               |
      |       See stsdas$copyright.stsdas for terms of use.        |
      |         For help, send e-mail to help@stsci.edu            |
      |                                                            |
      +------------------------------------------------------------+
stsdas/:
 analysis/      examples        hst_calib/      sobsolete/
 contrib/       fitsio/         playpen/        toolbox/
 describe       graphics/       problems


##  Define apertures and background sample in the database

We need to change the database of the aperture to define the center, lower and upper limit. Also to define the parameters of the background region to do the background substraction. This is the only way I have found to do it automatically. After this define the parameters in the .par file

In [6]:
copyfile(databasegeneral,filename)
#Now read and replace center and upper and lower values
with open(filename) as f:
    for lines in f:
        if 'image' in lines:
                imagenameor = lines
                imagename= lines.replace(lines.split()[1],sourcename)
        if 'center' in lines:
                numerocenteror = lines
                numerocenter = lines.replace(lines.split()[2], str(center))
        if 'low' in lines:
                numerolowor = lines
                numerolow = lines.replace(lines.split()[2],str(low))
        if 'high' in lines:
                numerohighor = lines
                numerohigh = lines.replace(lines.split()[2], str(high))
        if 'xmin' in lines:
            bloworiginal  = lines
            blow = lines.replace(lines.split()[1],str(min(b_low)))
        if 'xmax' in lines:
            buporiginal  = lines
            bup = lines.replace(lines.split()[1],str(max(b_up)))
        if 'sample' in lines:
            sampleor = lines
            sampleb = '\t\tsample '+str(min(b_low))+':'+str(max(b_low))+','+str(min(b_up))+':'+str(max(b_up))+'\n'
            break

with open(filename) as f:
    filedata = f.read()

filedata = filedata.replace(imagenameor,imagename)
filedata = filedata.replace(numerocenteror,numerocenter)
filedata = filedata.replace(numerolowor,numerolow)
filedata = filedata.replace(numerohighor,numerohigh)
filedata = filedata.replace(bloworiginal,blow)
filedata = filedata.replace(buporiginal,bup)
filedata = filedata.replace(sampleor,sampleb)

                                
with open(filename,'w') as f:
    f.write(filedata)

In [7]:
%%bash -s "$filename"
cat $1

# Tue 15:56:19 14-Mar-2017
begin	aperture cx967sexm 1 1250. 49.51036
	image	cx967sky
	aperture	1
	beam	1
	center	1250. 49
	low	-1249. -5
	high	1250. 5
	background
		xmin -13
		xmax 21
		function chebyshev
		order 1
		sample -13:-9,12:21
		naverage -3
		niterate 0
		low_reject 3.
		high_reject 3.
		grow 0.
	axis	2
	curve	5
		2.
		1.
		1.
		2500.
		0.



## Calling Apall

Now we define the parameters of apall, save it to the .par file and call the apall function.

In [8]:
if os.path.exists(sourcename+'.ms.fits'):
    os.remove(sourcename+'.ms.fits')
#Call them 
iraf.noao.twodspec()
iraf.noao.twodspec.apextract()
iraf.noao.twodspec.apextract.setParam('dispaxis','1')
#http://vivaldi.ll.iac.es/sieinvens/siepedia/pmwiki.php?n=HOWTOs.PythonianIRAF
iraf.noao.apextract.apall.setParam('input',sourcename+'.fits')
iraf.noao.apextract.apall.setParam('output',sourcename+'.ms.fits')
iraf.noao.twodspec.apextract.apall.setParam('recenter','no')
iraf.noao.twodspec.apextract.apall.setParam('resize','no')
iraf.noao.twodspec.apextract.apall.setParam('edit','no')
iraf.noao.twodspec.apextract.apall.setParam('trace','no')
iraf.noao.twodspec.apextract.apall.setParam('interactive','no')
iraf.noao.twodspec.apextract.apall.setParam('apertures','1')
iraf.noao.twodspec.apextract.apall.setParam('find','no')
iraf.noao.twodspec.apextract.apall.setParam('clean','yes')
iraf.noao.twodspec.apextract.apall.setParam('background','average')
iraf.noao.twodspec.apextract.apall.setParam('b_sample','-10,0:0,0')
iraf.noao.apextract.apall.saveParList(filename='uparm/'+sourcename+'.par')
iraf.noao.twodspec.apextract.apall(ParList='uparm/'+sourcename+'.par')

twodspec/:
 apextract/     longslit/
apextract/:
 apall          apedit          apflatten       apnormalize     apscatter
 apdefault@     apfind          apmask          aprecenter      apsum
 apdemos/       apfit           apnoise         apresize        aptrace


## Plotting with Bokeh

Plot the text file after doing dispcor and exporting the spectra to a text file using the iraf routine wspectext. 

In [9]:
#Plotting
##For srfm[0].header["CTYPE1"] = 'LINEAR'
#Other way
#srfm = fits.open(sourcename+'.ms.fits')
#secondstar = srfm[0].data#[0][0]##[0][0] if using clean
#secondstar = srfm[0].data[0][0]
#xn = srfm[0].header["NAXIS1"]
#refx = srfm[0].header["CRVAL1"]
#step = srfm[0].header['CD1_1']
#cr = srfm[0].header['CRPIX1']
#
#xlist = [ refx + step*(i - cr) for i in np.arange(1, len(secondstar)+1) ]
#Create ColumnDataSource
#x = np.array(xlist)
#y = np.array(secondstar)

#Not sure if I need to do this
if os.path.exists(sourcename+'.dispcor.fits'):
    os.remove(sourcename+'.dispcor.fits')

iraf.dispcor(sourcename+'.ms.fits',sourcename+'.dispcor.fits')
iraf.wspectext(sourcename+'.dispcor.fits[*,1,1]',sourcename+'.txt',header='no')

x=[]
y=[]
with open(sourcename+'.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))


source = ColumnDataSource(data=dict(x=x,y=y))
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra",
              active_scroll='wheel_zoom',plot_width=900, plot_height=700)
plot.add_tools(hover)
plot.line('x','y',source=source)
show(plot)

cx967sky.ms.fits: Resampling using current coordinate system
cx967sky.dispcor.fits: ap = 1, w1 =   3501.3, w2 =   9998.7, dw =      2.6, nw = 2500


## Normalizing the spectrum

We use the iraf routine continuum. We can define a sample to correctly normalize the desire range of wavelenght


In [10]:
sampler="6600:6800,7080:7140,7710:7870,8050:8090,8440:8910,8914:8916,8922:8926,8970:8973,8981:8983,8985:8987,9056:9059,9201:9204,9219:9221,9226:9228,9910:10400"

if os.path.exists(sourcename+'rn.fits'):
    os.remove(sourcename+'rn.fits')
    os.remove(sourcename+'bn.fits')
    os.remove(sourcename+'r.ms.fits')
    os.remove(sourcename+'b.ms.fits')
    os.remove(sourcename+'combn.fits')
    

tonormalize=sourcename+'.ms.fits'    
#Divide in red and blue
iraf.scopy(tonormalize,sourcename+'r.ms.fits',w1='7723',w2='INDEF')
iraf.scopy(tonormalize,sourcename+'b.ms.fits',w1='4750',w2='7565')
iraf.continuum(sourcename+'r.ms.fits',sourcename+'rn.fits',interactive='no',
               sample=sampler)
iraf.continuum(sourcename+'b.ms.fits',sourcename+'bn.fits',interactive='no',
               function='spline3',order='3',low_reject='3' ,high_reject='3')
iraf.scombine(sourcename+'*n.fits',sourcename+'combn.fits')

iraf.dispcor(sourcename+'combn.fits',listonly='yes')


iraf.wspectext(sourcename+'combn.fits', sourcename+'combn.txt',header='no')

xcontcx=[]
ycontcx=[]
with open(sourcename+'combn.txt') as f:
    for lines in f:
        xcontcx.append(float(lines.split()[0]))
        ycontcx.append(float(lines.split()[1]))

xcontcx = np.array(xcontcx)


Jan 25 19:44: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
   cx967skybn.fits[  1]
   cx967skyrn.fits[  1]

  Output image = cx967skycombn.fits, ncombine = 2
  w1 = 4750., w2 = 9996.1, dw = 2.599653, nw = 2019., dtype = -1
cx967skycombn.fits: Resampling using current coordinate system
cx967skycombn.fits: ap = 1, w1 =    4750., w2 =   9996.1, dw = 2.599653, nw = 2019


## Plotting the normalize spectra

We first define the desired range for the initial zoom:

In [11]:
#Plotting
xr = (8400,8900)
yr = (0.5,1.4)

"""
if os.path.exists(sourcename+'cont.dispcor.fits'):
    os.remove(sourcename+'cont.dispcor.fits')

iraf.dispcor(sourcename+'cont.fits',sourcename+'cont.dispcor.fits')
iraf.wspectext(sourcename+'cont.dispcor.fits[*,1,1]',sourcename+'cont.txt',header='no')

x=[]
y=[]
with open(sourcename+'cont.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))
"""
x = xcontcx
y = ycontcx
#
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

source = ColumnDataSource(data=dict(x=x,y=y))

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=700
             )
plot.add_tools(hover)
#plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)

GlyphRenderer(id='b4856dbf-ac61-4d10-a371-5c08bb1882dc', ...)

## Overplot lines

If desired we can define several know emission and absoprtion lines to overplot to the normalize spectra

In [12]:
##Emission Lines
class line(object):
    def __init__(self,name):
        self.name = name


diclines = {line('Ca II'):8498,line('Ca II'):8662,line('Ca II'):8541,
            line('O I'):8446,line('H'+u"\u03B1"):6563,line("(BaII, FeI and CaI)"):6497,
            line('O I Triplet(72,74,75)'):7774,line('He I'):7065,line('Fe I'):8621,
            line('P16'):8502,line('P15'):8545,line('P14'):8598,line('P13'):8665,
            line('P12'):8750,line('P11'):8863,line('P10'):9015,line('P9'):9229,
            line('P8'):9546,line('P17'):8467,line('P18'):8438,line('P19'):8413,line('P20'):8392}



for name, xloc in diclines.iteritems():
    yloc = y[np.where(abs(xloc - x) < 2)[0][0]]
    span = Arrow(end=NormalHead(fill_color='orange', size=10),
             x_start=xloc, y_start = yloc - .15, x_end = xloc, y_end= yloc-.03
            )
    plot.add_layout(span)
    my_label = Label(x=xloc, y=yloc-.03, text=name.name)
    plot.add_layout(my_label)

In [13]:
show(plot)

In [14]:
#Parametest for the fit:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx, array[idx]

def isDigit(x):
    try:
        float(x)
        return True
    except ValueError:
        return False
    
def gaussian(x, mu, sig,core):
    return core*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))




line = [8543,8598.]
regionfit = [[8520,8560],[8582,8609]]

#Initialize files log and lines
! echo '' > fited.log

for lines in zip(line,regionfit):
    regionf = "{} {}".format(lines[1][0],lines[1][1])
    #wavelenght
    xlimns = [find_nearest(x,i)[0] for i in lines[1]    ]
    wavex = x[xlimns[0]:xlimns[1]]
    lineszero = lines[0]
    ! echo '$lineszero' > lines.lines
    
    

    iraf.fitprofs(sourcename+'combn.fits',pos='lines.lines', reg=regionf ,
                  fitbackground= 'yes', 
                  logfile='fited.log')
                  #,nerrsample='100',sigma0='4',invgain='4')
    
    
    #Plotting the gaussian
    #Find in log file  
    npattern = re.compile('[-\d.]+')
    gparameters=[]
    with open('fited.log','r') as file:
        for lines in file:
            if '(' not in lines:
                temp = npattern.findall(lines)
            if len(temp) == 7 and all(isDigit(i) for i in temp):
                gparameters.append(temp)

    #gaussian
    gparamfinal = [ float(i) for i in gparameters[-1] ]
    centerg, contg, fluxg, eqwg, coreg, fwhmg, fwhml = gparamfinal
    yg = gaussian(wavex,centerg,fwhmg/2.3538,coreg) + contg

    #Plot
    #sourceg = ColumnDataSource(data=dict(x=wavex,y=yg))
    plot.line(wavex,yg,color='red')

show(plot)

# Jan 25 19:44 cx967skycombn.fits - Ap 1: MOS-GBS-917697
# Nfit=1, background=yes, positions=all, gfwhm=all, lfwhm=all
#   center      cont      flux       eqw      core     gfwhm     lfwhm
  8544.024 0.9927329  -1.83573     1.849 -0.128719      13.4        0.
# Jan 25 19:44 cx967skycombn.fits - Ap 1: MOS-GBS-917697
# Nfit=1, background=yes, positions=all, gfwhm=all, lfwhm=all
#   center      cont      flux       eqw      core     gfwhm     lfwhm
    8595.8  0.990842  -2.20484     2.225 -0.077184     26.84        0.


# The spectrum


Hard 

# Template

In [14]:
w1,w2,dw,nw= ['4750', '9993.499' ,'2.6', '2025']
#sep=[['INDEF','5770'],['5840','7565'],['7723','8540'],['8662','9150'],['9190','9300'],['9325','9460'],['9461','9610'],['9621','9764'],['9775','9920'],['9933','INDEF']]
sep=[['INDEF','5770'],['5840','7565'],['7723','8540'],['8662','9150'],['9190','9300'],['9325','9460'],['9480','9610'],['9621','9764'],['9775','9920'],['9933','INDEF']]

if os.path.exists(sourcename+'comb.fits'):
    os.remove(sourcename+'comb.fits')
for i,j in enumerate(sep):
    if os.path.exists(sourcename+'.'+str(i)+'tempms.fits'):
        os.remove(sourcename+'.'+str(i)+'tempms.fits')
    iraf.scopy(sourcename+'.ms.fits',sourcename+'.'+str(i)+'tempms.fits',w1=j[0],w2=j[1])
iraf.scombine(sourcename+'*tempms.fits*',sourcename+'comb.fits')

sampler="6600:6800,7080:7140,7710:7870,8050:8090,8440:8910,8914:8916,8922:8926,8970:8973,8981:8983,8985:8987,9056:9059,9201:9204,9219:9221,9226:9228,9910:10400"

if os.path.exists(sourcename+'rn.fits'):
    os.remove(sourcename+'rn.fits')
    os.remove(sourcename+'bn.fits')
    os.remove(sourcename+'r.ms.fits')
    os.remove(sourcename+'b.ms.fits')
    os.remove(sourcename+'combn.fits')
    
    
    
#Divide in red and blue
iraf.scopy(sourcename+'comb.fits',sourcename+'r.ms.fits',w1='7723',w2='INDEF')
iraf.scopy(sourcename+'comb.fits',sourcename+'b.ms.fits',w1='4750',w2='7565')
iraf.continuum(sourcename+'r.ms.fits',sourcename+'rn.fits',interactive='no',
               sample=sampler)
iraf.continuum(sourcename+'b.ms.fits',sourcename+'bn.fits',interactive='no',
               function='spline3',order='3',low_reject='3' ,high_reject='3')
iraf.scombine(sourcename+'*n.fits',sourcename+'combn.fits')

iraf.dispcor(sourcename+'combn.fits',listonly='yes')


iraf.wspectext(sourcename+'combn.fits', sourcename+'combn.txt',header='no')

xcontcx=[]
ycontcx=[]
with open(sourcename+'combn.txt') as f:
    for lines in f:
        xcontcx.append(float(lines.split()[0]))
        ycontcx.append(float(lines.split()[1]))

xcontcx = np.array(xcontcx)



#Dictionary of sources name and spectral type
dicsources={"hd162305":'A0V',"hd65810":'A1V', "Castor":"A2V",'hd211998':"A3V",
            "hd145689":'A4V',"hd39060":"A5V", "Altairus":'A7V', "hd26612":'A9V',
           "hd109931":'F0V',"hd40136":'F1V',"hd33256":"F2V","hd18692":"F3V",
           "hd37495":"F4V","hd16673":"F6V","hd45067":"F8V","hd10647":"F9V",
           "hd105113":"G0V","hd20807":"G1V","hd14802":"G2V","hd211415":"G3V",
           "hd59967":"G4V","hd59468":"G5V","hd140901":"G6V","hd25069":"G9V",
           "hd22049":"K2V","hd10361":"K5V","hd156274":"M0V","hd34055":"M6V",
           "hd148451":"G5III","hd37811":"G7III","hd45415":"G9III"}

#To ordered the tempaltes by their spectral type I had to do in the command line:
# mv hd187642.tfits ltairus.tfits
# mv hd60178.tfits Castor.tfits
iraf.image()
iraf.imfilter()

#Sort by spectral type:
dire = '../Templates/stars/' #Directory with all the template spectra
sortedlist = sorted(dicsources.items(), key=lambda t: t[1])
stars = [ i[0] for i in sortedlist]

#Can have one continuos color bar or a different one per spectral type
#colors = viridis(len(stars)) # colors map to plot
colors = Category20b_8 + Viridis8 + Dark2_8 + Paired8 + Set1_4



#plot parameters
xr = (7700,8900)
yr = (0.5,1.4)
p =  figure(x_axis_label='Angstrom', y_axis_label='Y',title="Click on the desired stellar template to overplot", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=1000
             )
p.line(x=xcontcx, y=ycontcx,color='firebrick',line_width=4,line_alpha=0.8,legend='CX Source')

hover2 = HoverTool(
        tooltips=[
            ("(x,y)", "($x{1}, $y)"),
        ]
    )
p.add_tools(hover2)


#Loop over the stars to create plot them on Bokeh plot
#This loops take time so only run if the final product is not available
#if want to delete them to rerun the loop change delete to True:
deletealldata = False
if deletealldata:
    if  os.path.exists(sourcename+'cont.dispcor.fits'):
        os.remove(sourcename+'cont.dispcor.fits')
    if  os.path.exists(name+'.fits'):
        os.remove(name+'.fits') 
    if  os.path.exists(sourcename+'cont.fits'):
        os.remove(sourcename+'cont.fits')
        
        
for index, star in enumerate(stars):
    
    #Copy star template to current folder
    ! cp {dire+star+'.fits'} {'temp{}.fits'.format(star)}
    name = star
    #Now apply the dispersion correction from the values defined before by looking at
    # at the values for the CX source. 
    if os.path.exists('temp{}.dispcor.fits'.format(star)):
        os.remove('temp{}.dispcor.fits'.format(star))

    #Ask about order
    #iraf.gauss('temp{}.fits'.format(star),'temp{}.fits'.format(star),sigma=10.)
    iraf.dispcor('temp{}.fits'.format(star),'temp{}.dispcor.fits'.format(star),w1=w1,w2=w2,dw=dw,nw=nw,verbose='no')
    iraf.gauss('temp{}.dispcor.fits'.format(star),'temp{}.dispcor.fits'.format(star),sigma=1.)
    #Normalize
    ##if it exist delte normalize spectrum:
    if os.path.exists('temp{}.dispcor.cont.fits'.format(star)):
        os.remove('temp{}.dispcor.cont.fits'.format(star))
        
    #Normalize and zero
    sourcename = star
    sep=[['INDEF','5770'],['5840','7565'],['7723','8540'],['8662','9150'],['9190','9300'],['9325','9460'],['9480','9610'],['9621','9764'],['9775','9920'],['9933','INDEF']]
    if os.path.exists(sourcename+'comb.fits'):
        os.remove(sourcename+'comb.fits')
    for i,j in enumerate(sep):
        if os.path.exists(sourcename+'.'+str(i)+'tempms.fits'):
            os.remove(sourcename+'.'+str(i)+'tempms.fits')
        iraf.scopy('temp{}.dispcor.fits'.format(star),sourcename+'.'+str(i)+'tempms.fits',w1=j[0],w2=j[1])
    iraf.scombine(sourcename+'*tempms.fits*',sourcename+'comb.fits')

    sampler="6600:6800,7080:7140,7710:7870,8050:8090,8440:8910,8914:8916,8922:8926,8970:8973,8981:8983,8985:8987,9056:9059,9201:9204,9219:9221,9226:9228,9910:10400"

    if os.path.exists(sourcename+'rn.fits'):
        os.remove(sourcename+'rn.fits')
        os.remove(sourcename+'bn.fits')
        os.remove(sourcename+'r.ms.fits')
        os.remove(sourcename+'b.ms.fits')
        os.remove(sourcename+'combn.fits')


    #Divide in red and blue
    iraf.scopy(sourcename+'comb.fits',sourcename+'r.ms.fits',w1='7723',w2='INDEF')
    iraf.scopy(sourcename+'comb.fits',sourcename+'b.ms.fits',w1='4750',w2='7565')
    iraf.continuum(sourcename+'r.ms.fits',sourcename+'rn.fits',interactive='no',
                   sample=sampler)
    iraf.continuum(sourcename+'b.ms.fits',sourcename+'bn.fits',interactive='no',
                   function='spline3',order='3',low_reject='3' ,high_reject='3')
    iraf.scombine(sourcename+'*n.fits',sourcename+'combn.fits')
        
    
    #Create a text file to plot the spectra
    iraf.wspectext(sourcename+'combn.fits','temp{}.cont.txt'.format(star),header='no')

    x=[]
    y=[]
    with open('temp{}.cont.txt'.format(star)) as f:
        for lines in f:
            x.append(float(lines.split()[0]))
            y.append(float(lines.split()[1]))
    #x = convolve(x, Gaussian1DKernel(stddev=10.))
    #If you want to have the name and not the spectra type change legend=name
    t = p.line(x=x,y=y,color=colors[index], line_alpha=0.0, 
           line_width=4,legend=dicsources[name],muted_alpha=1.,muted_color=colors[index])
    #This is a very ideficient way to have the colors in the legend to show up by default. 
    t2 = p.line(x=x[0],y=y[0],color=colors[index], line_alpha=1.0, 
           line_width=4,legend=dicsources[name],muted_alpha=1.,muted_color=colors[index])

    
p.legend.location = "top_left"
p.legend.click_policy="mute"
show(p)


Sep 14  1:50: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
  cx967sky.0tempms.fits[  1]
  cx967sky.1tempms.fits[  1]
  cx967sky.2tempms.fits[  1]
  cx967sky.3tempms.fits[  1]
  cx967sky.4tempms.fits[  1]
  cx967sky.5tempms.fits[  1]
  cx967sky.6tempms.fits[  1]
  cx967sky.7tempms.fits[  1]
  cx967sky.8tempms.fits[  1]
  cx967sky.9tempms.fits[  1]

  Output image = cx967skycomb.fits, ncombine = 10
  w1 = 3501.3, w2 = 9996.072, dw = 2.589622, nw = 2509., dtype = -1

Sep 14  1:50: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
   cx967skybn.fits[  1]
   cx967skyrn.fits[  1]

  Output image = cx967skycombn.fits, ncombine = 2
  w1 = 4750., w2 = 9993.483, dw = 2.589375, nw = 2026., dtype = -1
cx967skycombn.fits: Resampling using current coordinate system
cx967skycombn.fits: ap = 1, w1 =    4750., w2 = 9993.483, dw = 2.589375, nw = 2026
temphd162305.fits: Resampling usi


Sep 14  1:50: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
    hd26612bn.fits[  1]
    hd26612rn.fits[  1]

  Output image = hd26612combn.fits, ncombine = 2
  w1 = 4750., w2 = 9989.102, dw = 2.549441, nw = 2056., dtype = 0
temphd109931.fits: Resampling using current coordinate system
temphd109931.dispcor.fits: ap = 1, w1 =    4750., w2 =   9994.2, dw =      2.6, nw = 2018

Sep 14  1:50: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
  hd109931.0tempms.fits[  1]
  hd109931.1tempms.fits[  1]
  hd109931.2tempms.fits[  1]
  hd109931.3tempms.fits[  1]
  hd109931.4tempms.fits[  1]
  hd109931.5tempms.fits[  1]
  hd109931.6tempms.fits[  1]
  hd109931.7tempms.fits[  1]
  hd109931.8tempms.fits[  1]
  hd109931.9tempms.fits[  1]

  Output image = hd109931comb.fits, ncombine = 10
  w1 = 4750., w2 = 9991.65, dw = 2.549441, nw = 2057., dtype = 0

Sep 14  1:50: SCOMBINE
  combin


Sep 14  1:50: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
   hd105113bn.fits[  1]
   hd105113rn.fits[  1]

  Output image = hd105113combn.fits, ncombine = 2
  w1 = 4750., w2 = 9989.102, dw = 2.549441, nw = 2056., dtype = 0
temphd20807.fits: Resampling using current coordinate system
temphd20807.dispcor.fits: ap = 1, w1 =    4750., w2 =   9994.2, dw =      2.6, nw = 2018

Sep 14  1:50: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
  hd20807.0tempms.fits[  1]
  hd20807.1tempms.fits[  1]
  hd20807.2tempms.fits[  1]
  hd20807.3tempms.fits[  1]
  hd20807.4tempms.fits[  1]
  hd20807.5tempms.fits[  1]
  hd20807.6tempms.fits[  1]
  hd20807.7tempms.fits[  1]
  hd20807.8tempms.fits[  1]
  hd20807.9tempms.fits[  1]

  Output image = hd20807comb.fits, ncombine = 10
  w1 = 4750., w2 = 9991.65, dw = 2.549441, nw = 2057., dtype = 0

Sep 14  1:50: SCOMBINE
  combine = average,

                Images 
  hd45415.0tempms.fits[  1]
  hd45415.1tempms.fits[  1]
  hd45415.2tempms.fits[  1]
  hd45415.3tempms.fits[  1]
  hd45415.4tempms.fits[  1]
  hd45415.5tempms.fits[  1]
  hd45415.6tempms.fits[  1]
  hd45415.7tempms.fits[  1]
  hd45415.8tempms.fits[  1]
  hd45415.9tempms.fits[  1]

  Output image = hd45415comb.fits, ncombine = 10
  w1 = 4750., w2 = 9991.65, dw = 2.549441, nw = 2057., dtype = 0

Sep 14  1:50: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
    hd45415bn.fits[  1]
    hd45415rn.fits[  1]

  Output image = hd45415combn.fits, ncombine = 2
  w1 = 4750., w2 = 9989.102, dw = 2.549441, nw = 2056., dtype = 0
temphd25069.fits: Resampling using current coordinate system
temphd25069.dispcor.fits: ap = 1, w1 =    4750., w2 =   9994.2, dw =      2.6, nw = 2018

Sep 14  1:50: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
  hd25069.0tempms.fit

# Second Source

In [15]:
#----------#
#Variables-#
#----------#
sourcename = 'cx967sexm'
center = 49
low = -3
high = 3
b_low = [0,0]
b_up = [0,0]
databasegeneral = 'database/apcx967'
filename = 'database/ap'+sourcename
fitsfilename = 'mos_science_extracted_Q3.fits'
#----------#
#Exptime---#
#----------#
spectrawithsky = fits.open(fitsfilename)
exptime = spectrawithsky[0].header['EXPTIME']
iraf.stsdas()
iraf.images.imutil()
if os.path.exists(sourcename+'.fits'):
    os.remove(sourcename+'.fits')
    
iraf.images.imutil.imarith(fitsfilename,'*', exptime,sourcename)
#----------#
#Database--#
#----------#
copyfile(databasegeneral,filename)
#Now read and replace center and upper and lower values
with open(filename) as f:
    for lines in f:
        if 'image' in lines:
                imagenameor = lines
                imagename= lines.replace(lines.split()[1],sourcename)
        if 'center' in lines:
                numerocenteror = lines
                numerocenter = lines.replace(lines.split()[2], str(center))
        if 'low' in lines:
                numerolowor = lines
                numerolow = lines.replace(lines.split()[2],str(low))
        if 'high' in lines:
                numerohighor = lines
                numerohigh = lines.replace(lines.split()[2], str(high))
        if 'xmin' in lines:
            bloworiginal  = lines
            blow = lines.replace(lines.split()[1],str(min(b_low)))
        if 'xmax' in lines:
            buporiginal  = lines
            bup = lines.replace(lines.split()[1],str(max(b_up)))
        if 'sample' in lines:
            sampleor = lines
            sampleb = '\t\tsample '+str(min(b_low))+':'+str(max(b_low))+','+str(min(b_up))+':'+str(max(b_up))+'\n'
            break

with open(filename) as f:
    filedata = f.read()

filedata = filedata.replace(imagenameor,imagename)
filedata = filedata.replace(numerocenteror,numerocenter)
filedata = filedata.replace(numerolowor,numerolow)
filedata = filedata.replace(numerohighor,numerohigh)
filedata = filedata.replace(bloworiginal,blow)
filedata = filedata.replace(buporiginal,bup)
filedata = filedata.replace(sampleor,sampleb)

                                
with open(filename,'w') as f:
    f.write(filedata)

#----------#
#.par------#
#----------#
if os.path.exists(sourcename+'.ms.fits'):
    os.remove(sourcename+'.ms.fits')
#Call them 
iraf.noao.twodspec()
iraf.noao.twodspec.apextract()
iraf.noao.twodspec.apextract.setParam('dispaxis','1')
#http://vivaldi.ll.iac.es/sieinvens/siepedia/pmwiki.php?n=HOWTOs.PythonianIRAF
iraf.noao.apextract.apall.setParam('input',sourcename+'.fits')
iraf.noao.apextract.apall.setParam('output',sourcename+'.ms.fits')
iraf.noao.twodspec.apextract.apall.setParam('recenter','no')
iraf.noao.twodspec.apextract.apall.setParam('resize','no')
iraf.noao.twodspec.apextract.apall.setParam('edit','no')
iraf.noao.twodspec.apextract.apall.setParam('trace','no')
iraf.noao.twodspec.apextract.apall.setParam('interactive','no')
iraf.noao.twodspec.apextract.apall.setParam('apertures','1')
iraf.noao.twodspec.apextract.apall.setParam('find','no')
iraf.noao.twodspec.apextract.apall.setParam('clean','yes')
iraf.noao.twodspec.apextract.apall.setParam('background','none')
iraf.noao.twodspec.apextract.apall.setParam('b_sample','-10,0:0,0')
iraf.noao.apextract.apall.saveParList(filename='uparm/'+sourcename+'.par')
iraf.noao.twodspec.apextract.apall(ParList='uparm/'+sourcename+'.par')

IOError: [Errno 2] No such file or directory: 'database/apcx967'

In [ ]:
%%bash -s "$filename"
cat $1

## Plots

In [ ]:
#Not sure if I need to do this
if os.path.exists(sourcename+'.dispcor.fits'):
    os.remove(sourcename+'.dispcor.fits')

iraf.dispcor(sourcename+'.ms.fits',sourcename+'.dispcor.fits')
iraf.wspectext(sourcename+'.dispcor.fits[*,1,1]',sourcename+'.txt',header='no')

x=[]
y=[]
with open(sourcename+'.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))


source = ColumnDataSource(data=dict(x=x,y=y))
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra",
              active_scroll='wheel_zoom',plot_width=900, plot_height=700)
plot.add_tools(hover)
plot.line('x','y',source=source)
show(plot)

## Normalize


In [ ]:
sample = '5000:9000'
if os.path.exists(sourcename+'cont.fits'):
    os.remove(sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('input',sourcename+'.ms.fits')
iraf.noao.onedspec.continuum.setParam('output',sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('interactive','no')
iraf.noao.onedspec.continuum.setParam('sample',sample)
iraf.noao.onedspec.continuum.saveParList(filename='uparm/cont'+sourcename+'.par')
iraf.noao.onedspec.continuum(ParList='uparm/cont'+sourcename+'.par')

#Plotting
xr = (8400,8900)
yr = (0.7,1.1)

#Not sure if I need to do this
if os.path.exists(sourcename+'cont.dispcor.fits'):
    os.remove(sourcename+'cont.dispcor.fits')

iraf.dispcor(sourcename+'cont.fits',sourcename+'cont.dispcor.fits')
iraf.wspectext(sourcename+'cont.dispcor.fits[*,1,1]',sourcename+'cont.txt',header='no')

x=[]
y=[]
with open(sourcename+'cont.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))

x = np.array(x)
#
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

source = ColumnDataSource(data=dict(x=x,y=y))

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=700
             )
plot.add_tools(hover)
plot.line('x','y',source=source)
##Emission Lines
class line(object):
    def __init__(self,name):
        self.name = name


diclines = {line('Ca II'):8498,line('Ca II'):8662,line('Ca II'):8541,line('Fe I'):8621,line('Fe I'):8688,
            line('O I'):8446,line('Fe I'):8514,line('Fe I'):8468,
           line('H'+u"\u03B1"):6563,line("(BaII, FeI and CaI)"):6497,
            line('Ca I'):6162,line('Mg I'):8807}

for name, xloc in diclines.iteritems():
    yloc = y[np.where(abs(xloc - x) < 2)[0][0]]
    span = Arrow(end=NormalHead(fill_color='orange', size=10),
             x_start=xloc, y_start = yloc - .15, x_end = xloc, y_end= yloc-.03
            )
    plot.add_layout(span)
    my_label = Label(x=xloc, y=yloc-.03, text=name.name)
    plot.add_layout(my_label)
show(plot)

# References